In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
# %tensorflow_version 1.x
import os
os.chdir('/content/drive/MyDrive/')

In [ ]:
!ls

In [ ]:
#!unzip dataset.zip

In [ ]:
!ls

test  train  valid


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Chemin vers le dossier contenant les données
data_dir = 'Anarcade_project'

# Définir les chemins pour les ensembles d'entraînement, de test et de validation
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
valid_dir = os.path.join(data_dir, 'valid')

# Définir la taille des images et la taille du lot (batch size)
img_size = (224, 224)
batch_size = 32

# Normalisation des pixels entre 0 et 1
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Charger les données d'entraînement
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',  # Si vous avez plus de deux classes, utilisez 'categorical'
    classes=['c1', 'c3']  # Les noms des sous-dossiers
)

# Charger les données de validation
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    classes=['c1', 'c3']
)

# Vérifier les étiquettes associées aux classes
print(train_generator.class_indices)
print(valid_generator.class_indices)


Found 7291 images belonging to 2 classes.
Found 2430 images belonging to 2 classes.
{'c1': 0, 'c3': 1}
{'c1': 0, 'c3': 1}


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Créer un modèle séquentiel
model = Sequential()

# Ajouter une couche de convolution avec activation ReLU et une couche de pooling
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Ajouter une deuxième couche de convolution avec activation ReLU et une couche de pooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Ajouter une couche de flatten pour convertir la sortie en vecteur
model.add(Flatten())

# Ajouter une couche dense avec activation ReLU
model.add(Dense(128, activation='relu'))

# Ajouter la couche de sortie avec activation sigmoid (binaire) ou softmax (multiclasse)
model.add(Dense(1, activation='sigmoid'))

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Afficher la structure du modèle
model.summary()

# Entraîner le modèle
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,  # ajustez le nombre d'époques selon vos besoins
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size
)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [ ]:
# Évaluer la performance du modèle sur les données de validation
evaluation = model.evaluate(valid_generator, steps=valid_generator.samples // batch_size)

# Afficher les métriques d'évaluation
print("Loss:", evaluation[0])
print("Accuracy:", evaluation[1])


75/75 [==============================] - 74s 984ms/step - loss: 0.9648 - accuracy: 0.7771
Loss: 0.9648328423500061
Accuracy: 0.7770833373069763


In [ ]:
# Save the trained model
model.save('Anarcade_project/model')

#

In [21]:
from tensorflow.keras.models import save_model

# Assuming 'model' is the name of your trained model
# Save the model in HDF5 format
model.save('/Anarcade_project/my_model.h5')


In [ ]:
!pip freeze > requirements.txt
